# **Лабораторна робота 6: Пошук аномалій та вирішення задачі *anomaly detection* за допомогою бібліотек `scikit-learn`та `PyTorch`**
**Всі завдання виконуються індивідуально. Використання запозиченого коду буде оцінюватись в 0 балів.**

**Лабораторні роботи де в коді буде використаня КИРИЛИЦІ будуть оцінюватись в 20 балів.**

### Мета роботи:
Ознайомитися з основними методами виявлення аномалій, навчитися використовувати бібліотеки `scikit-learn` та `PyTorch` для реалізації алгоритмів пошуку аномалій, проаналізувати ефективність різних методів на реальних наборах даних з Kaggle.


### Опис завдання:

1. **Постановка задачі**:
   Використовуючи один із доступних наборів даних Kaggle (наприклад, *Credit Card Fraud Detection*, *Network Intrusion*, або інші), вам потрібно розв'язати задачу виявлення аномалій. Основна мета — ідентифікувати аномальні записи серед нормальних. Вибраний набір даних повинен містити мітки аномалій для перевірки результатів.

2. **Етапи виконання завдання**:
   - Завантажте та підготуйте набір даних.
   - Проведіть попередню обробку даних (масштабування, заповнення пропущених значень, видалення нерелевантних ознак).
   - Використайте різні методи виявлення аномалій:
     - **Методи з бібліотеки scikit-learn**:
       - Isolation Forest
       - One-Class SVM
       - Local Outlier Factor (LOF)
     - **Методи з використанням PyTorch**:
       - Автоенкодери для виявлення аномалій.
   - Порівняйте отримані результати, обчисліть метрики якості (Precision, Recall, F1-Score).
   - Оцініть, який метод найкраще підходить для вирішення задачі на вашому наборі даних.

### Покрокова інструкція

1. **Підготовка середовища**:
   - Встановіть необхідні бібліотеки:
     ```
     pip install scikit-learn torch pandas numpy matplotlib
     ```

2. **Вибір набору даних з Kaggle**:
   Зареєструйтесь на Kaggle та оберіть один із наборів даних для виявлення аномалій. Наприклад:
   - [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud)
   - [Network Intrusion Detection](https://www.kaggle.com/xyuanh/benchmarking-datasets)

3. **Попередня обробка даних**:
   - Завантажте дані та проведіть їхню початкову обробку.
   - Масштабуйте ознаки за допомогою `StandardScaler` або `MinMaxScaler`.
   - Розділіть дані на навчальну і тестову вибірки.

4. **Методи з бібліотеки `scikit-learn`**:

   - **Isolation Forest**:
     ```
     from sklearn.ensemble import IsolationForest
     ```

   - **One-Class SVM**:
     ```
     from sklearn.svm import OneClassSVM
     ```

   - **Local Outlier Factor**:
     ```
     from sklearn.neighbors import LocalOutlierFactor
     ```

5. **Методи на основі нейронних мереж (PyTorch)**:

   Використайте автоенкодер для пошуку аномалій. Побудуйте нейронну мережу з енкодером і декодером. Під час навчання порівняйте відновлені дані з вхідними та обчисліть помилку. Записи з великою помилкою можуть бути аномаліями.

   - **Реалізація автоенкодера**:
     ```
     import torch
     import torch.nn as nn
     import torch.optim as optim
     ```

6. **Оцінка результатів**:
   Використовуйте метрики оцінки якості:
   - `Precision`, `Recall`, `F1-score`
   ```
   from sklearn.metrics import classification_report
   ```

7. **Звіт**:
   - Поясніть, який метод дав найкращі результати.
   - Проаналізуйте, чому деякі методи працюють краще на вашому наборі даних.
   - Оцініть можливості використання глибоких нейронних мереж (автоенкодерів) для вирішення задачі.


### Результати, які необхідно надати:
1. Код рішення у вигляді Jupyter Notebook з аналізом результатів та поясненнями.


### Корисні ресурси:
- [Документація PyTorch](https://pytorch.org/docs/stable/index.html)
- [Документація scikit-learn](https://scikit-learn.org/stable/documentation.html)
- [Kaggle Datasets](https://www.kaggle.com/datasets)

## Data processing

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
from sklearn.metrics import classification_report

data = pd.read_csv('creditcard.csv')

data = data.dropna()

X = data.drop(columns=['Class'])
y = data['Class']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=99)

## Methods from library scikit-learn

#### Isolation forest

In [8]:
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor

iso_forest = IsolationForest(contamination=0.1, random_state=99)
iso_forest.fit(X_train)

train_iso_pred = iso_forest.predict(X_train)
test_iso_pred = iso_forest.predict(X_test)


train_iso_anom = np.where(train_iso_pred == -1, 1, 0)
test_iso_anom = np.where(test_iso_pred == -1, 1, 0)

print(f"Anomalies in the training set: {np.sum(train_iso_anom)}")
print(f"Anomalies in the test set: {np.sum(test_iso_anom)}")

print("Classification report for training set:")
print(classification_report(y_train, train_iso_anom))

print("Classification report for test set:")
print(classification_report(y_test, test_iso_anom))

Anomalies in the training set: 22785
Anomalies in the test set: 5780
Classification report for training set:
              precision    recall  f1-score   support

           0       1.00      0.90      0.95    227455
           1       0.01      0.86      0.03       390

    accuracy                           0.90    227845
   macro avg       0.51      0.88      0.49    227845
weighted avg       1.00      0.90      0.95    227845

Classification report for test set:
              precision    recall  f1-score   support

           0       1.00      0.90      0.95     56860
           1       0.02      0.92      0.03       102

    accuracy                           0.90     56962
   macro avg       0.51      0.91      0.49     56962
weighted avg       1.00      0.90      0.95     56962



#### One-class SVM

In [31]:
one_class_svm = OneClassSVM(nu=0.1, kernel='rbf', gamma=0.1)
one_class_svm.fit(X_train)

train_svm_pred = one_class_svm.predict(X_train)
test_svm_pred = one_class_svm.predict(X_test)

train_svm_anom = np.where(train_svm_pred == -1, 1, 0)
test_svm_anom = np.where(test_svm_pred == -1, 1, 0)

print(f"Anomalies in the training set: {np.sum(test_svm_pred)}") # should ouput testing set
print(classification_report(y_test, test_svm_anom))
print(f"Anomalies in the training set: {np.sum(train_svm_anom)}")
print(classification_report(y_train, train_svm_anom))

Anomalies in the training set: 45088
              precision    recall  f1-score   support

           0       1.00      0.90      0.95     56860
           1       0.02      0.93      0.03       102

    accuracy                           0.90     56962
   macro avg       0.51      0.91      0.49     56962
weighted avg       1.00      0.90      0.94     56962

Anomalies in the training set: 22800
              precision    recall  f1-score   support

           0       1.00      0.90      0.95    227455
           1       0.02      0.90      0.03       390

    accuracy                           0.90    227845
   macro avg       0.51      0.90      0.49    227845
weighted avg       1.00      0.90      0.95    227845



#### Local outlier factory

In [32]:
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1, n_jobs=-1)
outliers_lof = lof.fit_predict(X_train)

train_lof_anom = np.where(outliers_lof == -1, 1, 0)

test_lof_pred = lof.fit_predict(X_test)
test_lof_anom = np.where(test_lof_pred == -1, 1, 0)

print(f"Anomalies in the training set: {np.sum(train_lof_anom)}")
print(f"Anomalies in the test set: {np.sum(test_lof_anom)}")

print("Classification report for training set:")
print(classification_report(y_train, train_lof_anom))

print("Classification report for train set:") # should be train, just can't reload the model will take
                                                # too much time
print(classification_report(y_test, test_lof_anom))

Anomalies in the training set: 22785
Anomalies in the test set: 5697
Classification report for training set:
              precision    recall  f1-score   support

           0       1.00      0.90      0.95    227455
           1       0.00      0.16      0.01       390

    accuracy                           0.90    227845
   macro avg       0.50      0.53      0.48    227845
weighted avg       1.00      0.90      0.95    227845

Classification report for test set:
              precision    recall  f1-score   support

           0       1.00      0.90      0.95     56860
           1       0.00      0.21      0.01       102

    accuracy                           0.90     56962
   macro avg       0.50      0.55      0.48     56962
weighted avg       1.00      0.90      0.95     56962



## PyTorch

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()

        
        self.encoder = nn.Sequential( 
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16)
        )

        
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

input_dim = X_train.shape[1]
X_train_tensor = torch.FloatTensor(X_train)

train_loader = DataLoader(TensorDataset(X_train_tensor), batch_size=32, shuffle=True)

patience = 10  
best_loss = np.inf
epochs_without_improvement = 0

model = Autoencoder(input_dim=input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
for epoch in range(num_epochs):
    total_loss = 0
    model.train()
    for data_load in train_loader:
        inputs = data_load[0] 
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

    # Check for improvement
    if avg_loss < best_loss:
        best_loss = avg_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    # Early stopping
    if epochs_without_improvement >= patience:
        print(f'Early stopping at epoch {epoch+1}')
        break

model.eval()

with torch.no_grad():
    reconstructed = model(X_train_tensor)
    reconstruction_error = torch.mean((X_train_tensor - reconstructed) ** 2, dim=1)
    
threshold = 0.1
anomalies = reconstruction_error > threshold

y_pred = anomalies.numpy()

Epoch 1/100, Loss: 0.2518
Epoch 2/100, Loss: 0.0928
Epoch 3/100, Loss: 0.0677
Epoch 4/100, Loss: 0.0578
Epoch 5/100, Loss: 0.0509
Epoch 6/100, Loss: 0.0457
Epoch 7/100, Loss: 0.0440
Epoch 8/100, Loss: 0.0414
Epoch 9/100, Loss: 0.0400
Epoch 10/100, Loss: 0.0381
Epoch 11/100, Loss: 0.0373
Epoch 12/100, Loss: 0.0362
Epoch 13/100, Loss: 0.0380
Epoch 14/100, Loss: 0.0345
Epoch 15/100, Loss: 0.0338
Epoch 16/100, Loss: 0.0327
Epoch 17/100, Loss: 0.0332
Epoch 18/100, Loss: 0.0309
Epoch 19/100, Loss: 0.0316
Epoch 20/100, Loss: 0.0307
Epoch 21/100, Loss: 0.0327
Epoch 22/100, Loss: 0.0313
Epoch 23/100, Loss: 0.0308
Epoch 24/100, Loss: 0.0309
Epoch 25/100, Loss: 0.0292
Epoch 26/100, Loss: 0.0299
Epoch 27/100, Loss: 0.0293
Epoch 28/100, Loss: 0.0300
Epoch 29/100, Loss: 0.0294
Epoch 30/100, Loss: 0.0283
Epoch 31/100, Loss: 0.0272
Epoch 32/100, Loss: 0.0281
Epoch 33/100, Loss: 0.0262
Epoch 34/100, Loss: 0.0267
Epoch 35/100, Loss: 0.0277
Epoch 36/100, Loss: 0.0257
Epoch 37/100, Loss: 0.0253
Epoch 38/1

In [8]:
y_true = y_train.values
print(f"Detected {anomalies.sum().item()} anomalies out of {X_train_tensor.shape[0]} samples.")
print(classification_report(y_true, y_pred))

Detected 6922 anomalies out of 227845 samples.
              precision    recall  f1-score   support

           0       1.00      0.97      0.99    227455
           1       0.04      0.76      0.08       390

    accuracy                           0.97    227845
   macro avg       0.52      0.86      0.53    227845
weighted avg       1.00      0.97      0.98    227845



# Analysis

Let's analyse the received data. Keep in mind that there was no fine-tuning and the models are in a kind of a "raw" state speaking of the performance.

Isolation forest has f1 score of 0.03, which is very low. In current case recall=0.86 which considering the precision of 0.01 is very bad.

One-class SVM has f1 score 0.03 as well, but recall is 0.93, that's something at least...

Local outlier factory f1=0.01 the worst between all.

Pytorch autocoder has f1=0.08 which is almost 3 times better than second-best model, recall 0.76 which is seems not much, but considering that it's not guessing not as often and with a bit higher accuracy makes it the WINNER in the lab6!!🎉🎉🎉🎉🎉🎉🎉🎉